In [1]:
import pandas as pd

# 파일 로드
file_path = 'data\mortgage_customer_churn_dataset.csv'
data = pd.read_csv(file_path)

# 컬럼명만 가져오기
columns = data.columns

# 특정 구간 슬라이싱 (예: 'REGION_CLIENT_W_CITY'부터 'YEARS_BEGINEXPLUATATION_AVG'까지)
start_col = 'REGION_RATING_CLIENT_W_CITY'
end_col = 'YEARS_BEGINEXPLUATATION_AVG'
sliced_columns = columns[columns.get_loc(start_col) : columns.get_loc(end_col) + 1]

# 결과
print(sliced_columns)


Index(['REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION',
       'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BEGINEXPLUATATION_AVG'],
      dtype='object')


In [ ]:
118 * 0.2 * 0.2

4.720000000000001

In [11]:
pip install ace_tools

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# import ace_tools as tools 
# summary = {}

# for col in df.columns:
#     if pd.api.types.is_numeric_dtype(df[col]):  # 숫자형 데이터 타입만 처리
#         summary[col] = {
#             'min': df[col].min(),
#             'max': df[col].max(),
#             'mean': df[col].mean(),
#             'mode': df[col].mode().tolist(),  # 최빈값
#             'null_count': df[col].isnull().sum()
#         }
#     else:  # 문자열 데이터 타입 처리 (min, max, mean 제외)
#         summary[col] = {
#             'mode': df[col].mode().tolist(),
#             'null_count': df[col].isnull().sum()
#         }

# # 결과를 데이터프레임으로 변환하여 표시
# summary_df = pd.DataFrame(summary).T
# tools.display_dataframe_to_user(name="DataFrame Summary", dataframe=summary_df)
# pd.set_option('display.max_columns', None) 
# pd.set_option('display.max_rows', None) 
# summary_df = pd.DataFrame(summary).T
# display(summary_df)

In [21]:
import pandas as pd

# CSV 파일을 DataFrame으로 불러오기
data = pd.read_csv('data\mortgage_customer_churn_dataset.csv')

# 모든 행과 열을 출력하도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# DataFrame 출력
print(data)


KeyboardInterrupt: 

In [22]:


summary = {}

for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):  # 숫자형 데이터 타입만 처리
        summary[col] = {
            'min': df[col].min(),
            'max': df[col].max(),
            'mean': df[col].mean(),
            'mode': df[col].mode().tolist(),  # 최빈값
            'null_count': df[col].isnull().sum()
        }
    else:  # 문자열 데이터 타입 처리 (min, max, mean 제외)
        summary[col] = {
            'mode': df[col].mode().tolist(),
            'null_count': df[col].isnull().sum()
        }

# 결과를 데이터프레임으로 변환하여 표시
summary_df = pd.DataFrame(summary).T

# 전체 출력 옵션 설정
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 

# DataFrame 표시
display(summary_df)


,min,max,mean,mode,null_count
SK_ID_CURR,100002,456255,278180.518577,"[100002, 100003, 100004, 100006, 100007, 10000...",0
TARGET,0,1,0.080729,[0],0
NAME_CONTRACT_TYPE,NaN,NaN,NaN,[Cash loans],0
CODE_GENDER,NaN,NaN,NaN,[F],0
FLAG_OWN_CAR,NaN,NaN,NaN,[N],0
FLAG_OWN_REALTY,NaN,NaN,NaN,[Y],0
CNT_CHILDREN,0,19,0.417052,[0],0
AMT_INCOME_TOTAL,25650.0,117000000.0,168797.919297,[135000.0],0
AMT_CREDIT,45000.0,4050000.0,599025.999706,[450000.0],0
AMT_GOODS_PRICE,40500.0,4050000.0,538396.207429,[450000.0],278
